Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 3: ES

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The goal of the game is to **avoid** taking the last object.

* Task2.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task2.2: An agent using evolved rules using ES

## Instructions

* Create the directory `lab2` inside your personal course repository for the course 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.



In [3]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
import random
from copy import deepcopy


## The *Nim* and *Nimply* classes

In [4]:
Nimply = namedtuple("Nimply", "row, num_objects")


In [5]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects


## Sample (and silly) startegies 

In [6]:
def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)


In [7]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))


In [8]:
def adaptive(state: Nim) -> Nimply:
    """A strategy that can adapt its parameters"""
    genome = {"love_small": 0.5}


In [9]:
import numpy as np


def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)


def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked


def optimal(state: Nim) -> Nimply:
    analysis = analize(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    ply = random.choice(spicy_moves)
    return ply


## Oversimplified match

In [10]:
logging.getLogger().setLevel(logging.INFO)

strategy = (optimal, pure_random)

nim = Nim(5)
logging.info(f"init : {nim}")
player = 0
while nim:
    ply = strategy[player](nim)
    logging.info(f"ply: player {player} plays {ply}")
    nim.nimming(ply)
    logging.info(f"status: {nim}")
    player = 1 - player
logging.info(f"status: Player {player} won!")


INFO:root:init : <1 3 5 7 9>
INFO:root:ply: player 0 plays Nimply(row=1, num_objects=1)
INFO:root:status: <1 2 5 7 9>
INFO:root:ply: player 1 plays Nimply(row=4, num_objects=7)
INFO:root:status: <1 2 5 7 2>
INFO:root:ply: player 0 plays Nimply(row=3, num_objects=5)
INFO:root:status: <1 2 5 2 2>
INFO:root:ply: player 1 plays Nimply(row=3, num_objects=1)
INFO:root:status: <1 2 5 1 2>
INFO:root:ply: player 0 plays Nimply(row=3, num_objects=1)
INFO:root:status: <1 2 5 0 2>
INFO:root:ply: player 1 plays Nimply(row=2, num_objects=3)
INFO:root:status: <1 2 2 0 2>
INFO:root:ply: player 0 plays Nimply(row=4, num_objects=2)
INFO:root:status: <1 2 2 0 0>
INFO:root:ply: player 1 plays Nimply(row=1, num_objects=2)
INFO:root:status: <1 0 2 0 0>
INFO:root:ply: player 0 plays Nimply(row=2, num_objects=2)
INFO:root:status: <1 0 0 0 0>
INFO:root:ply: player 1 plays Nimply(row=0, num_objects=1)
INFO:root:status: <0 0 0 0 0>
INFO:root:status: Player 0 won!


## Genetic Strategies

### Utils

In [11]:
def game(size:int = 5, strat:callable = optimal) -> (bool, int):  # plays a game of given size using strategy strat against an opponent who randomly picks their move
    strategy = (strat, pure_random) if random.random() < 0.5 else (pure_random, strat)  # randomly select player 1 and player 2
    nim = Nim(size)
    player = 0
    num_moves = 0
    while nim:
        num_moves += 1
        ply = strategy[player](nim)
        nim.nimming(ply)
        player = 1 - player
    return (strategy[player] == strat, num_moves)

In [12]:
class Individual:
    genotype:float
    fitness: tuple

    def phenotype(self, state: Nim) -> Nimply:
        analysis = analize(state)
        logging.debug(f"analysis:\n{pformat(analysis)}")
        spicy_moves = [(ply, ns) for ply, ns in analysis["possible_moves"].items() if ns != 0]
        if not spicy_moves:
            spicy_moves = list(analysis["possible_moves"].keys())
            return random.choice(spicy_moves)
        if random.random() < self.genotype: # choose the move with the smallest non-zero nim sum
            spicy_moves.sort(key = lambda i : i[1]) # sort moves by their nim sum
            return spicy_moves[0][0]
        return random.choice(spicy_moves)[0]

In [13]:
def fitness(individual:Individual):  # the fitness is calculated as the winrate and the average number of moves calculated over 100 games
    winrate = 0
    tot_moves = 0.0
    for i in range(100):
        won, moves = game(strat=individual.phenotype)
        winrate += won
        tot_moves += moves
    return winrate, - tot_moves/100

In [14]:
ind = Individual()

ind.genotype = random.random()
ind.fitness = fitness(Individual)

print(ind.genotype, ind.fitness)

TypeError: Individual.phenotype() missing 1 required positional argument: 'state'